# Tagfilter am Beispiel von Freiberg, Sachsen

In diesem Notebook wird gezeigt, wie der Stadtgraph zerklüftet, wenn er nach bestimmten Tags gefiltert wird.

Um einen Zusammenhängenden Graphen zu erhalten müssen die fehlenden Daten ergänzt werden.

Mehr Informationen zu den Beschreibungen der Szenarien finden Sie hier. [Beschreibung Szenarien](beschreibung-szenarien)

In [ ]:
import osmnx as ox
from configobj import ConfigObj
import pyproj # we need this for jupyter-book to build right...
from pathlib import Path
from helpers import configreader as cfr
from helpers import osmhelper as oh
from helpers import ready4robots as r4r

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
cfg = ConfigObj(r"notebook.ini")

In [ ]:
def analyse_scenario(city, cfg, scenario, override=False, with_speed=True, folder=None):

    simplify = cfr.get_simplify(cfg)

    # load city
    G_nx = oh.get_place(placename=city, simplify=simplify, graph_paths=["osm"],
                        clean_periphery=True,
                        file_identifier="", retain_all=True)

    # filter_graph by scenario
    filter_scenario = r4r.get_scenario_filter(cfg, scenario["name"])

    H_nx = G_nx.copy()
    H_nx = oh.filter_graph_by_dict(H_nx, filter_scenario, drop_nodes=False)

    return H_nx

In [ ]:
scenarios = cfr.get_scenarios(cfg)
graphs = []
cities = cfr.get_cities(cfg)
city = cities[0]
for scenario in scenarios:
    # print(scenario.name)
    graph = analyse_scenario(city, cfg, scenario, override=True)
    graphs.append(graph)

In [ ]:
for graph, scenario in zip(graphs, scenarios):
    print(scenario["name"])
    x = ox.plot_graph(graph, figsize=(12, 12), node_size=0)

In [ ]:
G_nx = oh.get_place(placename=city, simplify=True, graph_paths=[Path("..", "osm").resolve()],
                    clean_periphery=True,
                    file_identifier="", retain_all=True, ox_cache=Path("..", "cache").resolve())

stats = oh.tag_stats(G_nx)

stats

In [ ]:
ox.plot_graph_folium(graph)

In [ ]:
nodes, edges = ox.graph_to_gdfs(graph)
nodes.explore()

In [ ]:
edges.explore()